## <span style="color:grey">Initialization</span>

In [58]:
import pandas as pd
import numpy as np
import warnings
import time
import datetime
import logging
import copy 
import os
from multiprocessing import Process
warnings.filterwarnings('ignore')

exchange = 'NSE'
symbol = "HDFC"

today = datetime.datetime.now()

if today.weekday() >= 5:
    today = today - datetime.timedelta(days = today.weekday() - 4)

#today = datetime.datetime(2019, 4, 8, 15, 45, 0)

#configuration for downloading data
days =  1
years = 5
toDate = today
fromDate = toDate - datetime.timedelta(days = days)
freq = '1min'

mode = "Algo" # Backtesting/Live/Algo
offline = False
if mode =="Live":
    offline = False

KiteAPIKey = "b2w0sfnr1zr92nxm"
KiteAPISecret = "jtga2mp2e5fn29h8w0pe2kb722g3dh1q"

from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 0

%run "KiteConnect_Library.ipynb"    

### Kite - OAuth Login

In [2]:
#logger.setFormatter("%(asctime)s:%(levelname)s: %(message)s")

In [3]:
from kiteconnect import KiteConnect
import platform
from selenium import webdriver
import re

logging.basicConfig(filename="log/live_log.log", filemode="a", level=logging.DEBUG, format="[%(asctime)s: %(levelname)s]:%(message)s")
logger=logging.getLogger()

kite = KiteConnect(api_key=KiteAPIKey)
reauthentication = False

f = open("kite_data/access_token.txt", mode="r")
access_token = f.readlines()
logger.info(access_token[0])
kite.set_access_token(access_token[0])

try:
    logger.info("Welcome "+kite.profile()['user_name'])
except:
    if platform.system() == "Windows":
        driver = webdriver.Chrome("./automation/chromedriver.exe")
    else:
        driver = webdriver.Chrome("./automation/chromedriver")
    driver.implicitly_wait(60)
    driver.get(kite.login_url())
    request_token = re.findall("request_token=(.*)",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    access_token = data["access_token"]
    kite.set_access_token(access_token)
    f = open("kite_data/access_token.txt", mode="w")
    f.write(access_token)
    f.close()
    #request_token = re.findall("request_token=(.*)&action",driver.current_url)
    reauthentication = True

    #data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    #access_token = data["access_token"]
    #kite.set_access_token(access_token)
    #f = open("kite_data/access_token.txt", mode="w")
    #f.write(access_token)
    #f.close()

In [4]:
if reauthentication:
    request_token = re.findall("request_token=(.*)&action",driver.current_url)

    data = kite.generate_session(request_token[0], api_secret=KiteAPISecret)
    access_token = data["access_token"]
    kite.set_access_token(access_token)
    f = open("kite_data/access_token.txt", mode="w")
    f.write(access_token)
    f.close()

### Download NSE Stock Master List

In [5]:
if offline != True:
    instruments_df = getInstruments(exchange)
    instruments_df.to_hdf('kite_data/kite_data.h5', key=exchange, mode='w')

instruments_df = pd.read_hdf('kite_data/kite_data.h5', key=exchange, mode='r')

EQSYMBOL = lambda x:instruments_df[instruments_df['instrument_token']==x].index[0]
EQTOKEN = lambda x:instruments_df.loc[x,'instrument_token']

### Filter Stocks - portfolio maker

In [12]:
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

downloadlist = nifty50['Symbol']
industry = niftynext50['Industry'].unique()

In [13]:
portfolio = pd.DataFrame(downloadlist.tolist())

In [14]:
portfolio = pd.DataFrame(['TCS', 'RIIL', 'HDFC','NDTV'])

#portfolio = pd.DataFrame(['BAJAJ-AUTO','CIPLA','HINDUNILVR','M&M','WIPRO','SBIN','RIIL','TCS','INFY'])

#portfolio = pd.DataFrame(downloadlist)
#portfolio = pd.DataFrame(['BAJAJ-AUTO','CIPLA'])

#portfolio = pd.DataFrame(['BAJAJ-AUTO'])
portfolioToken = portfolio[0].apply(EQTOKEN)

In [15]:
portfolio.tail()

,0
0,TCS
1,RIIL
2,HDFC
3,NDTV


### Download Historical Data - Equity

In [16]:
def portfolioDownload(stocklist, toDate):
    stocklist_df = pd.DataFrame()
    for index, row in stocklist.iterrows():
        symbol = row[0]
        logging.info("Downloading data for: "+symbol)
        temp_data = downloadData(symbol,  toDate - dt.timedelta(days = 1), toDate)
        temp_data['symbol'] = symbol
        temp_data.set_index(['symbol',temp_data.index], inplace=True)
        #print(temp_data)
        stocklist_df = stocklist_df.append(temp_data)
    
    #print(stocklist_df)
    return stocklist_df    

if offline != True:
    portfolio_df = pd.DataFrame()
    portfolio_df = portfolioDownload(portfolio, today)
    portfolio_df.to_hdf('kite_data/kite_data.h5', key="portfolio",mode='a')

#portfolio_df.loc['HDFC']

### Batch Downloader

In [17]:
def batchDownload(downloadlist):
    for symbol in downloadlist:
        logging.info("Downloading data for: "+symbol)

        raw_data = downloadData(symbol, fromDate, toDate)
        raw_data_day = downloadData(symbol,  toDate - dt.timedelta(days = years * 365),toDate, freq="day")

        raw_data_day.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
        raw_data.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')

### Incremental Download - untested

In [18]:
def incrementalDownload(downloadlist):
    for symbol in downloadlist:
        logging.info("Downloading data for: "+symbol)
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol,mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate)
        tempData = tempData.append(raw_data)
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol,mode='a')
        
        tempData = pd.read_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='r')
        fromDate = tempData.index[-1]
        toDate = datetime.datetime.now()
        raw_data = downloadData(symbol, fromDate, toDate, freq="day")
        
        tempData.to_hdf('kite_data/kite_data.h5', key=symbol+"day",mode='a')
    
    

### Historical Data storage strategy

* minute level data and day level ohlc data is stored in the hd5
* symbol name is used as a key in the hd5 file system

### Tick Data storage strategy
- Tick data should not be merged with the downloaded historical data
- At the start of the trading session, last 60 candles from the historical data is fetched and stored in the dataframe for holding live data
- Two DataFrames ares created for storing live data: 1 for storing timestamp and LTP, another for OHLC
- OHLC data is created by grouping data first based on stock symbol and then based on timestamp(seconds and miliseconds are ignored)
- Streaming data from all the stocks in portfolio are strored in a single table. During post-procession subset of the master table sliced based on symbol is used
- Streaming data is resampled for minute frequency every minute to convert it to OHLC data which is stored in another dataframe
- Symbol is used as a key for the streaming OHLC dataframe storage

### Load Data from disk

In [18]:
if mode=="Live":
    raw_data = pd.read_hdf('kite_data/kite_data.h5', key="portfolio",mode='r')
else:
    raw_data = pd.read_hdf('kite_data/kite_data.h5', key="portfolio",mode='r')

##  <span style="color:green">Trading Strategies </span>

In [66]:
#pivotPoint(raw_data, pos=1, plot=False)
#raw_data = calculateStats(raw_data)
#raw_data = emasma(raw_data)
#raw_data = adx(raw_data)
#raw_data = detectCDPattern(raw_data,strPlot="hammer", plot=True)
#raw_data = stoch(raw_data)

# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0
iMACD = 0
iMACDhist = 0
iMACDsignal = 0
UP = 0
DOWN = 1
SD = 0
iRSI = 0 
atr = 0

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a, days: ROC(a,days) >= 0.01
TREND_DOWN = lambda a, days: ROC(a,days) <= -0.01
CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        global iMACD, iMACDhist, iMACDsignal, SD, iRSI
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,2,2,1)
        AroonDown, AroonUp = AROON(HIGH, LOW, 25)
        iMACD, iMACDsignal, iMACDhist = MACDEXT(CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        SD = STDDEV(CLOSE)
        iRSI = RSI(CLOSE, timeperiod=14)
        #atr = ATR(CLOSE)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( (REF(CLOSE, 1) < REF(BBB, 1)) & (CLOSE > BBB), columns=["buy"] )
    
    def long_bull_engulf_ema(self):
        return pd.DataFrame( (EMA(CLOSE, 9) < EMA(CLOSE, 21)) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100) ,columns =["buy"])
    
    def long_hammer(self):
        return pd.DataFrame(TREND_DOWN(CLOSE, 10) & (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) != 0), columns=["buy"])
    
    def long_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(A,B) ,columns=["buy"])
    
    def long_aroon(self):
        return pd.DataFrame((AroonUp > 50) & (AroonDown<50), columns=["buy"])
        
    
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame((REF(CLOSE,1) > REF(BBT,1)) & (CLOSE<BBT), columns=["sell"])
    
    def short_bear_engulf(self):
        return pd.DataFrame(TREND_UP(CLOSE,10) & (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100),columns=["sell"])
    
    def short_hanging_man(self):
        return pd.DataFrame(
            TREND_UP(CLOSE, 10) &
            (CDLHANGINGMAN(OPEN, HIGH, LOW, CLOSE) == -100)
            , columns=["sell"])
    
    def short_bear_kicking(self):
        return pd.DataFrame(CDLKICKING(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
        
    def short_shooting_star(self):
        return pd.DataFrame( TREND_UP(CLOSE, 5) & CDLSHOOTINGSTAR(OPEN, HIGH, LOW, CLOSE) != 0, columns=["sell"])
    
    def short_ichimoku(self):
        C = ( SMA(HIGH, 9) + SMA(LOW, 9) )/2
        D = ( SMA(HIGH, 26) + SMA(LOW, 26) )/2
        A = (C+D)/2
        B = (SMA(HIGH,52)+SMA(LOW,52))/2
        
        return pd.DataFrame( CROSSOVER(B,A) ,columns=["sell"])
    
    def short_aroon(self):
        return pd.DataFrame((AroonUp < 50) & (AroonDown > 50), columns=["sell"])
    
    def long_macd_crossover(self):
        return pd.DataFrame(CROSSOVER(iMACD, iMACDsignal), columns=["buy"])
    
    
    def short_macd_crossover(self):
        return pd.DataFrame(CROSSOVER(iMACDsignal, iMACD), columns=["sell"])
    
    def tradeDecision(self):
        buy  = self.long_macd_crossover() # | self.long_bb() #| self.long_ichimoku() #| self.long_bull_engulf_ema() #| self.long_ichimoku()
        
        sell = self.short_bb() # |self.short_macd_crossover() # | self.short_bb() #| self.short_bear_engulf() #| self.short_hanging_man() #|  self.short_ichimoku()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = sell['close'] = CLOSE
        buy['sl'] = CLOSE - SD
        sell['sl'] = CLOSE + SD
        buy['so'] = CLOSE + 4 * SD
        sell['so'] = CLOSE - 4 * SD

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        return (buy,sell)
    
    def tradeRecommendation(self):
        buy, sell = self.tradeDecision()
        timeNow = dt.datetime.now().replace(second=0, microsecond=0)
        delT1 = timeNow - datetime.timedelta(minutes = 1)
        delT2 = timeNow - datetime.timedelta(minutes = 2)
        logging.debug(str(timeNow))
        logging.debug(buy.tail(3))
        logging.debug(sell.tail(3))
        if (delT1 == buy.index[-1]) | (delT2 == buy.index[-1]) :
            return "buy,"+str(timeNow) +","+str(buy.iloc[-1]['low'])+","+str(buy.iloc[-1]['close'])+"\n"
        elif (delT1 == sell.index[-1]) | (delT2 == sell.index[-1]) :  
            return "sell,"+str(timeNow) +","+str(sell.iloc[-1]['high'])+","+str(sell.iloc[-1]['close'])+"\n"
        return ""

if mode=="Algo":
    global fig
    #noofcandles = 50
    x = "RIIL"
    temp_data = downloadData(x, fromDate = toDate - dt.timedelta(days=2), toDate=toDate, freq="minute")
    
    logger.debug(temp_data)
    #temp_data = downloadData(x, fromDate, toDate, "day")
    #logging.debug(temp_data)
        
    #yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    #yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    #xMin = temp_data.index[-1*noofcandles]
    #xMax = temp_data.index[-1]
    fig = createPlot(x)
    #fig['layout']['yaxis']['range'] = [yMin, yMax]
    #fig['layout']['yaxis']['range'] = [xMin, xMax]

    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    temp_data = aroon(temp_data,5, True)
    #obv(temp_data,2, True)
    
    #print(temp_data.tail(200))
    
    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    #print(buy)
    #print(sell)
    #print(CLOSE)
    
    #a.tradeRecommendation()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, False)


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



In [57]:
from talib import MACDEXT
help(MACDEXT)

Help on function MACDEXT in module talib._ta_lib:

MACDEXT(...)
    MACDEXT(real[, fastperiod=?, fastmatype=?, slowperiod=?, slowmatype=?, signalperiod=?, signalmatype=?])
    
    MACD with controllable MA type (Momentum Indicators)
    
    Inputs:
        real: (any ndarray)
    Parameters:
        fastperiod: 12
        fastmatype: 0
        slowperiod: 26
        slowmatype: 0
        signalperiod: 9
        signalmatype: 0
    Outputs:
        macd
        macdsignal
        macdhist



In [23]:
temp_data = downloadData('HDFC', fromDate = toDate - dt.timedelta(days=365), toDate= toDate, freq="day")

In [21]:
temp_data.head(50)

,close,high,low,open,volume,bbt,bbm,bbb,macd,macdsignal,macdhist,RSI,aroondown,aroonup,obv
date,,,,,,,,,,,,,,,
2019-04-11 09:15:00,2022.65,2031.45,2021.90,2028.00,58346,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58346.0
2019-04-11 09:16:00,2021.10,2025.15,2021.05,2023.75,25880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84226.0
2019-04-11 09:17:00,2018.60,2021.70,2018.60,2021.20,15366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99592.0
2019-04-11 09:18:00,2022.00,2022.30,2018.60,2018.60,11542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111134.0
2019-04-11 09:19:00,2021.00,2022.50,2021.00,2022.00,11031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122165.0
2019-04-11 09:20:00,2024.40,2025.00,2021.00,2021.00,9534,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131699.0
2019-04-11 09:21:00,2025.30,2028.95,2024.75,2025.20,10744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,142443.0
2019-04-11 09:22:00,2025.80,2025.80,2024.10,2025.30,3677,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,146120.0
2019-04-11 09:23:00,2026.30,2026.85,2025.50,2025.50,5223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151343.0


In [34]:
temp_data['atr'] = ATR(temp_data.high, temp_data.low, temp_data.close)
#temp_data['sma'] = SMA(temp_data.close)
temp_data['sd-'] = temp_data.shift(1).close - STDDEV(temp_data.shift(1).close)
temp_data['sd+'] = temp_data.shift(1).close + STDDEV(temp_data.shift(1).close)
temp_data['2sd-'] = temp_data.shift(1).close - 2*STDDEV(temp_data.shift(1).close)
temp_data['2sd+'] = temp_data.shift(1).close + 2* STDDEV(temp_data.shift(1).close)

from talib import TRANGE, NATR

temp_data['tr'] = TRANGE(temp_data.high, temp_data.low, temp_data.close)
temp_data['natr'] = NATR(temp_data.high, temp_data.low, temp_data.close)

In [35]:
temp_data.tail(10)

,close,high,low,open,volume,atr,sd-,sd+,2sd-,2sd+,tr,natr
date,,,,,,,,,,,,
2019-04-01,1959.65,1975.00,1945.05,1968.1,3328552,33.956042,1952.933688,1983.566312,1937.617377,1998.882623,29.95,1.732761
2019-04-02,1995.95,2001.00,1955.00,1966.1,3655232,34.816325,1943.215481,1976.084519,1926.780963,1992.519037,46.00,1.744349
2019-04-03,2013.10,2032.45,1997.15,2004.0,4345772,34.936587,1970.720842,2021.179158,1945.491685,2046.408315,36.50,1.735462
2019-04-04,2042.05,2058.80,2006.30,2008.0,4463212,36.191117,1988.213023,2037.986977,1963.326047,2062.873953,52.50,1.772293
2019-04-05,2059.20,2065.00,2035.00,2045.0,2576789,35.748894,2012.036003,2072.063997,1982.022007,2102.077993,30.00,1.736057
2019-04-08,2054.90,2072.50,2029.25,2068.0,2057768,36.284687,2024.241805,2094.158195,1989.283610,2129.116390,43.25,1.765764
2019-04-09,2069.15,2073.00,2039.15,2062.5,2381281,36.110781,2030.337195,2079.462805,2005.774390,2104.025610,33.85,1.745199
2019-04-10,2029.25,2068.00,2025.10,2067.5,3036356,36.677868,2049.794004,2088.505996,2030.438007,2107.861993,44.05,1.807459
2019-04-11,2022.80,2048.00,2017.50,2028.0,2521729,36.236592,2015.357578,2043.142422,2001.465155,2057.034845,30.50,1.791408


## <span style="color:blue">BackTesting</span>

In [24]:
logger.setLevel(logging.DEBUG)

### Multithreaded

In [28]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
from multiprocessing import Pool
def threadedBatchAnalysis(x):
    fig = ""
    temp_data = downloadData(x, fromDate=toDate - dt.timedelta(days=365), toDate=toDate, freq="day")
    #logging.debug(temp_data)
        
    yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
    yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()

    xMin = temp_data.index[-1*noofcandles]
    xMax = temp_data.index[-1]
    fig = createPlot(x)
    fig['layout']['yaxis']['range'] = [yMin, yMax]
    fig['layout']['yaxis']['range'] = [xMin, xMax]

    #print(temp_data.head())
    #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
    temp_data = candlestick(temp_data,1,True)
    temp_data = bbands(temp_data,1, True)

    temp_data = macd(temp_data,3,True)
    temp_data = rsi(temp_data,4, True)
    temp_data = aroon(temp_data,5, True)
    obv(temp_data,2, True)

    a = algoTrade(temp_data)
    buy, sell = a.tradeDecision()
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    #print(os.getcwd())
    plotData()

if __name__ == '__main__':
    #if mode == "Algo":
    p = Pool(5)
    p.map_async(threadedBatchAnalysis, portfolio[0])
    #p.apply_async(threadedBatchAnalysis, ['SBIN'])
    #portfolio
    
#threadedBatchAnalysis()

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



In [31]:
import gc
gc.get_count()

(50, 4, 2)

In [30]:
gc.collect()

633

### Backtesting - Single Threaded

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

noofcandles = 50
def batchAnalysis(portfolio=portfolio):
    global fig, noofcandles
    noofcandles = 50
    
    for index, row in portfolio.iterrows():
        fig = ""
        symbol = row[0] #row.iloc[:,0]
        temp_data = downloadData(symbol, fromDate, toDate, "day")
        
        #temp_data['symbol'] = symbol
        #temp_data.set_index(['temp_data.index], inplace=True)
        logging.debug(temp_data)
        
        yMin = temp_data.iloc[-1*noofcandles:-1]['low'].min()-10
        yMax = temp_data.iloc[-1*noofcandles:-1]['high'].max()
        
        xMin = temp_data.index[-1*noofcandles]
        xMax = temp_data.index[-1]
        fig = createPlot(symbol)
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['yaxis']['range'] = [xMin, xMax]

        #temp_data = resample2(temp_data.loc[symbol],"1min").dropna()[-20*noofcandles:-1]
        temp_data = candlestick(temp_data,1,True)
        temp_data = bbands(temp_data,1, True)

        temp_data = macd(temp_data,3,True)
        temp_data = rsi(temp_data,4, True)
        temp_data = aroon(temp_data,5, True)
        obv(temp_data,2, True)
        
        a = algoTrade(temp_data)
        buy, sell = a.tradeDecision()
        buy_df = annotateBuySell(buy, "Buy")
        sell_df = annotateBuySell(sell, "Sell")
        
        plotData(symbol)
        #input("Press Enter to continue")
    

#batchAnalysis()

## <span style="color:red"> Live Trading</span>

### Order Management

In [19]:
def buy(symbol, price, stoploss,quantity=1): 
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                trigger_price=stoploss,
                                price=price,
                                variety=kite.VARIETY_REGULAR
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
        
def buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_BUY,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                stoploss=stoploss,
                                trigger_price=trigger,
                                price=price,
                                squareoff=squareoff,
                                variety=kite.VARIETY_BO
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))


def sell(symbol, price, stoploss, quantity=1): 
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                            exchange=kite.EXCHANGE_NSE,
                            transaction_type=kite.TRANSACTION_TYPE_SELL,
                            quantity=quantity,
                            order_type=kite.ORDER_TYPE_SL,
                            product=kite.PRODUCT_MIS,
                            trigger_price=stoploss,
                            price=price,
                            variety=kite.VARIETY_REGULAR)
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))

def sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol,
                                exchange=kite.EXCHANGE_NSE,
                                transaction_type=kite.TRANSACTION_TYPE_SELL,
                                quantity=quantity,
                                order_type=kite.ORDER_TYPE_SL,
                                product=kite.PRODUCT_MIS,
                                stoploss=stoploss,
                                trigger_price=trigger,
                                price=price,
                                squareoff=squareoff,
                                variety=kite.VARIETY_BO
                                )
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
        
def getOrders():    
    # Fetch all orders
    return pd.DataFrame(kite.orders())

def cancelOrder(orderId):
    try:
        kite.cancel_order(variety=kite.VARIETY_REGULAR, order_id=orderId, parent_order_id=None)    
    except Exception as e:
        logging.info("Order Cancellation failed: {}".format(e.message))

In [68]:
if mode == "Live" & False:
    buy_bo("WIPRO",283.1,283.1, 0.65,0.25)

In [36]:
if mode == "Live" & False:
    sell("WIPRO",283,285)

There is an order update
There is an order update


In [69]:
if mode == "Live" & False:
    cancelOrder(190412002140702)

In [20]:
if mode == "Live" & False:
    positions_df = pd.DataFrame(kite.positions()['net'])
    positions_df

,average_price,buy_m2m,buy_price,buy_quantity,buy_value,close_price,day_buy_price,day_buy_quantity,day_buy_value,day_sell_price,...,product,quantity,realised,sell_m2m,sell_price,sell_quantity,sell_value,tradingsymbol,unrealised,value
0,0,283.4,283.4,1,283.4,0,283.4,1,283.4,283.9,...,BO,0,0,283.9,283.9,1,283.9,WIPRO,0.5,0.5


In [21]:
if mode == "Live" & False:
    orders_df = getOrders()
    if orders_df.empty != True:
        orders_df[orders_df['status']!=''][['order_id','exchange','tradingsymbol','transaction_type','price','trigger_price', 'quantity', 'order_type', 'product', 'status']]
    



In [ ]:
if mode == "Live" & False:
    kite.modify_order(variety=kite.VARIETY_REGULAR, 
                  order_id=190411002378318, quantity=1, price=282, 
                  order_type=kite.ORDER_TYPE_SL)



In [24]:
logger.setLevel(logging.INFO)

### WebSocket: Tick Handler and sampler

In [53]:
import datetime

def resample(ws, freq="1min"):
    F = open("kite_data/recommendation.csv","a") 
    
    logging.debug(str(ws.prevtimeStamp)+": In resampler function")
    
    if ws.LiveStream.empty:
        logging.debug(str(ws.prevtimeStamp)+": Empty dataframe, Exiting resampler")
        return
      
    LiveStream2 = ws.LiveStream.groupby(['symbol','date']).agg({'price':['first','max','min','last'], 'volume':['last']})
    LiveStream2.columns = LiveStream2.columns.droplevel()
    LiveStream2.columns = ['open', 'high','low','close', 'volume']

    for index, data in LiveStream2.groupby(level=0):
        sampled = data.loc[index].resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'volume':{'volume':'last'}})
        sampled.columns = sampled.columns.droplevel()

        sampled['symbol'] = index
        sampled.set_index(['symbol',sampled.index], inplace=True)

        ws.LiveStreamOHLC = ws.LiveStreamOHLC.append(sampled)
        
    #logger.info(index+":\n"+str(ws.LiveStreamOHLC.loc[portfolio.iloc[-1,0]].tail(240)))

    ws.LiveStreamOHLC.to_csv("kite_data/livestreamohlc.csv", mode='a')

    a = algoTrade(ws.LiveStreamOHLC.loc[portfolio.iloc[-1,0]].tail(120))
    reco_df = a.tradeRecommendation()
    
    try:
        if reco_df != "":
            F.write(reco_df)
    except   Exception as e:
        logging.critical("Not able to resample and analyse data"+str(e))
    finally:
        F.close()
        



def ticksHandler(ws, ticks):
    timeStamp = dt.datetime.now().replace(second=0, microsecond=0)
    tick_df = pd.DataFrame(ticks)
    
    #logging.debug("Prev: "+str(ws.prevtimeStamp)+"\nNow: "+str(timeStamp))
    
    try:
        tick_df.loc[tick_df['timestamp'].isna(), 'timestamp'] = timeStamp
        tick_df = tick_df[['timestamp','instrument_token','last_price','volume']]
        tick_df.instrument_token = tick_df.instrument_token.apply(EQSYMBOL)
        tick_df.columns = ['date','symbol','price','volume']
        tick_df.set_index(['symbol','date'], inplace=True)
        
        timeStamp = tick_df.index[0][-1].to_pydatetime()
        
        #logging.info(str(timeStamp - ws.prevtimeStamp))
        #print(type(timeStamp))
        
        #logging.debug("Exception: ticksHandler: "+str(tick_df))
        #timeStamp = tick_df.loc[portfolio.iloc[-1,0]].index[-1]
    except  Exception as e:
        logging.debug("Exception: ticksHandler: "+str(e)+str(tick_df))
        
    if( (timeStamp - ws.prevtimeStamp) >= datetime.timedelta(minutes=1)):
        ws.prevtimeStamp = timeStamp
        
        #ws.cbufferSampling = ws.cbufferAccum.copy(deep=True) #Copy accumulated stream to sampling buffer
        #del ws.cbufferAccum
        #gc.collect()
        #ws.cbufferAccum = pd.DataFrame() #Reset accumulation buffer
        #logging.debug(ws.cbufferSampling)
        resample(ws)
    
    #ws.cbufferAccum = ws.cbufferAccum.append(tick_df)
    
    ws.LiveStream = ws.LiveStream.append(tick_df)
    #ws.LiveStream.to_csv("kite_data/livestream.csv", mode='a')

def orderNotification(ws,data):
    logger.debug(data)

def initTrade(ws):
    ws.prevtimeStamp = datetime.datetime.now() - dt.timedelta(minutes=10)
    toDate = datetime.datetime.now()
    #ws.cbufferAccum = pd.DataFrame()
    #ws.cbufferSampling = pd.DataFrame()
    ws.LiveStream = pd.DataFrame()
    ws.LiveStreamOHLC = pd.DataFrame()
    ws.LiveStreamOHLC = portfolioDownload(portfolio, toDate)
    

### WebSocket: Kite Integration

In [20]:
#!python
from kiteconnect import KiteTicker

#logging.basicConfig(level=logging.CRITICAL)
logger.setLevel(logging.DEBUG)


# Initialise
kws = KiteTicker(KiteAPIKey, kite.access_token)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    #logging.debug("Ticks: {}".format(ticks))
    ticksHandler(ws, ticks)


def on_connect(ws, response):
    initTrade(ws)
    
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe(portfolioToken.tolist())

    ws.subscribe([225537])
    
    # Set RELIANCE to tick in `full` mode.
    # MODE_LTP, MODE_QUOTE, or MODE_FULL

    ws.set_mode(ws.MODE_FULL, portfolioToken.tolist())
    ws.set_mode(ws.MODE_FULL, [225537]) 
    #ws.set_mode(ws.MODE_LTP, [225537, 3861249]) 
    #ws.set_mode(ws.MODE_MODE_QUOTE, [2714625,779521]) 

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

def on_order_update(ws, data):
    logger.info("New Order Update")
    orderNotification(ws,data)

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_order_update = on_order_update

#kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.

### WebSocket: Start/Stop

In [21]:
if mode == "Live":
    kws.connect(threaded=True)

In [56]:
if mode == "Live":
    kws.close()

ERROR:kiteconnect.ticker:Connection closed: None - None


In [28]:
if mode == "Live":
    kws.is_connected()

True

### Even wait

In [43]:
import asyncio
async def waiter(event):
    print('waiting for it ...')
    await event.wait()
    print('... got it!')

async def main():
    # Create an Event object.
    event = asyncio.Event()

    # Spawn a Task to wait until 'event' is set.
    waiter_task = asyncio.create_task(waiter(event))

    # Sleep for 1 second and set the event.
    await asyncio.sleep(1)
    event.set()

    # Wait until the waiter task is finished.
    await waiter_task

await main()

waiting for it ...
... got it!
